In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler , MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score,roc_auc_score,confusion_matrix, classification_report
from sklearn.model_selection import KFold, cross_val_score, train_test_split,GridSearchCV

In [25]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [26]:
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization, ReLU,Input,Dropout

In [27]:
from scikeras.wrappers import KerasClassifier    #for scikit wrapper
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

In [46]:
from sklearn.naive_bayes import GaussianNB

# Data Preprocessing 

In [29]:
df = pd.read_csv('force dataset grasp classification.csv')
print(df.head())

   Time [s]  Thumb  Index  Middle   Ring  Pinky             Object Grasp  \
0   331.982  0.290 -0.043   0.141 -0.164  0.948  Skillet lid (T01)  Hook   
1   332.482  0.284 -0.023   0.133 -0.176  0.944  Skillet lid (T01)  Hook   
2   332.982  0.294 -0.012   0.145 -0.195  0.948  Skillet lid (T01)  Hook   
3   333.482  0.284 -0.012   0.141 -0.199  0.940  Skillet lid (T01)  Hook   
4   333.982  0.288 -0.004   0.156 -0.188  0.936  Skillet lid (T01)  Hook   

   Mass(g)  
0   220.03  
1   220.03  
2   220.03  
3   220.03  
4   220.03  


In [30]:
newdf = df[['Thumb','Index','Middle','Ring','Pinky','Object','Grasp']]
newdf

,Thumb,Index,Middle,Ring,Pinky,Object,Grasp
0,0.290,-0.043,0.141,-0.164,0.948,Skillet lid (T01),Hook
1,0.284,-0.023,0.133,-0.176,0.944,Skillet lid (T01),Hook
2,0.294,-0.012,0.145,-0.195,0.948,Skillet lid (T01),Hook
3,0.284,-0.012,0.141,-0.199,0.940,Skillet lid (T01),Hook
4,0.288,-0.004,0.156,-0.188,0.936,Skillet lid (T01),Hook
...,...,...,...,...,...,...,...
16951,-0.040,-0.040,0.090,0.110,0.120,Washer (T26),Pulp Pinch (PP)
16952,-0.020,-0.050,0.110,0.120,0.100,Washer (T26),Pulp Pinch (PP)
16953,-0.010,-0.060,0.110,0.130,0.110,Washer (T26),Pulp Pinch (PP)
16954,-0.010,-0.060,0.110,0.120,0.120,Washer (T26),Pulp Pinch (PP)


In [31]:
newdf.Object.value_counts().size

24

In [32]:
grasp_object_counts = newdf.groupby(['Grasp', 'Object']).size()

# Display the result
print(grasp_object_counts)

Grasp                       Object                
 Diagonal Volar Grip (DVG)  Screwdriver (T06)          700
                            Skillet (T24)              719
                            Spatula (T15)              584
Cylindrical Grip (CG)       Chips Can (T05)            808
                            Coffee Can (T14)           869
                            Power Drill (T23)          890
Extension Grip (EG)         Craker Box (T13)           844
                            Plate (T04)               1044
                            Tetra Pack (T22)           651
Hook                        Coffee Cup (T19)           957
                            Pitcher base (T10)         894
                            Skillet lid (T01)          476
Lateral Pinch (LP)          Bowl (T07)                 630
                            Key (T25)                  344
                            XS Clamp (T16)             608
Pulp Pinch (PP)             Plastic Peer (T17)         515
     

In [33]:
test_objects = ['Spatula (T15)','Coffee Can (T14)','Tetra Pack (T22)','Skillet lid (T01)','Key (T25)','Small Marker (T08)','Mini Soccer ball (T11)','Large Marker (T03)']
len(test_objects)

8

In [34]:
# print(newdf.shape)
# training_data = newdf[~newdf['Object'].isin(test_objects)]
# test_data = newdf[newdf['Object'].isin(test_objects)]
# print(training_data.shape)
# print(test_data.shape)

In [35]:
# trainy = training_data.Grasp
# trainx = training_data.drop("Grasp",axis='columns')
# testy = test_data.Grasp
# testx = test_data.drop("Grasp",axis='columns')
# print(trainx)
# print(trainy)

In [36]:
y = newdf.Grasp
x= newdf.drop("Grasp",axis='columns')
print(x)
print(y)

       Thumb  Index  Middle   Ring  Pinky             Object
0      0.290 -0.043   0.141 -0.164  0.948  Skillet lid (T01)
1      0.284 -0.023   0.133 -0.176  0.944  Skillet lid (T01)
2      0.294 -0.012   0.145 -0.195  0.948  Skillet lid (T01)
3      0.284 -0.012   0.141 -0.199  0.940  Skillet lid (T01)
4      0.288 -0.004   0.156 -0.188  0.936  Skillet lid (T01)
...      ...    ...     ...    ...    ...                ...
16951 -0.040 -0.040   0.090  0.110  0.120      Washer (T26) 
16952 -0.020 -0.050   0.110  0.120  0.100      Washer (T26) 
16953 -0.010 -0.060   0.110  0.130  0.110      Washer (T26) 
16954 -0.010 -0.060   0.110  0.120  0.120      Washer (T26) 
16955 -0.010 -0.050   0.110  0.120  0.120      Washer (T26) 

[16956 rows x 6 columns]
0                   Hook
1                   Hook
2                   Hook
3                   Hook
4                   Hook
              ...       
16951    Pulp Pinch (PP)
16952    Pulp Pinch (PP)
16953    Pulp Pinch (PP)
16954    Pulp Pin

In [37]:
trainx,testx,trainy,testy = train_test_split(x,y,random_state=0,test_size=0.3)
#dividing train and test randomly
#if scaler is used before train test split, data leakage will happen because scaler will also be using information from 
#the then unsplit test set for scaling.
print(trainx)
print(trainy)

       Thumb  Index  Middle  Ring  Pinky               Object
13375   3.78   1.04    0.46  0.75   1.56        Spatula (T15)
8710    1.70   1.55    2.58  1.80   0.17     Craker Box (T13)
1662    0.35   0.66    0.14 -0.57  -0.70  Pitcher base (T10) 
7765    0.13   0.29    0.13  0.21   1.18         Plate (T04) 
13572   0.06   0.00    0.00  0.05   0.10        Skillet (T24)
...      ...    ...     ...   ...    ...                  ...
9225   -0.09  -0.15    0.12  0.00   0.39     Tetra Pack (T22)
13123   3.66   0.82    0.45  0.68   1.69        Spatula (T15)
9845    1.85   1.96    2.02  2.41   0.20      Chips Can (T05)
10799   3.61   1.94    1.14  2.18   2.24     Coffee Can (T14)
2732    3.70   2.35    2.56  2.90   2.33          Apple (T02)

[11869 rows x 6 columns]
13375     Diagonal Volar Grip (DVG)
8710            Extension Grip (EG)
1662                           Hook
7765            Extension Grip (EG)
13572     Diagonal Volar Grip (DVG)
                    ...            
9225          

In [38]:
numerical_cols = ['Thumb','Index','Middle','Ring','Pinky']
categorical_cols = ['Object']

In [39]:
def preprocess_xxtest(x,xtest):
    scaler = StandardScaler()
    x[numerical_cols] = scaler.fit_transform(x[numerical_cols])
    xtest[numerical_cols] = scaler.transform(xtest[numerical_cols])
    x = x.drop("Object",axis='columns')
    xtest = xtest.drop("Object",axis='columns')
    return x.to_numpy(),xtest.to_numpy()

def preprocess_y(y):
    return pd.get_dummies(y).to_numpy()

def preprocess_xxtest_dl(x,xtest):
    scaler = MinMaxScaler()      #only change is here. This scaler should not be used when there are outliers.
    x[numerical_cols] = scaler.fit_transform(x[numerical_cols])
    xtest[numerical_cols] = scaler.transform(xtest[numerical_cols])
    x = x.drop("Object",axis='columns')
    xtest = xtest.drop("Object",axis='columns')
    return x.to_numpy(),xtest.to_numpy()


In [40]:
from sklearn.preprocessing import LabelEncoder
trainy_label = trainy
testy_label = testy
encode = LabelEncoder()
trainy_label = encode.fit_transform(trainy_label)
testy_label = encode.transform(testy_label)
print(trainy_label)
print(testy_label)

[0 2 3 ... 1 1 6]
[2 1 4 ... 7 6 1]


In [41]:
dltrainx,dltestx = preprocess_xxtest_dl(trainx,testx)
trainx,testx = preprocess_xxtest(trainx,testx)
trainy_hot = preprocess_y(trainy)
testy_hot = preprocess_y(testy)
print(trainx)
print(testx)

[[ 1.0690581  -0.14819259 -0.5149372  -0.15136836  0.61250092]
 [-0.05530088  0.2294691   1.24824179  0.76343371 -0.52233255]
 [-0.78505311 -0.42958757 -0.78107743 -1.30140526 -1.232624  ]
 ...
 [ 0.0257827   0.53307947  0.78249639  1.29489015 -0.49783974]
 [ 0.97716337  0.51826921  0.05061077  1.09450494  1.16767125]
 [ 1.02581352  0.82187958  1.23160802  1.72179779  1.24114968]]
[[-0.08773431  0.51826921  1.65576901  1.26875295 -0.00798357]
 [ 0.30687244  0.85150011 -0.53157097  0.34523848 -0.6692894 ]
 [-0.92559798 -0.89611083 -0.97236572 -0.82222322 -0.53866109]
 ...
 [ 0.50147304  0.41459737  0.34170165 -0.82222322 -0.82441052]
 [-0.64450823 -0.68136203 -0.58978914 -0.88321002 -1.02035299]
 [ 0.52309532  0.71080262  0.79913016  1.39943896 -0.13861188]]


In [42]:
print(dltrainx)
print(dltestx)
print(trainy_hot)

[[0.38322487 0.36666667 0.19674355 0.40316901 0.38435754]
 [0.23282719 0.4152381  0.4843962  0.58802817 0.22905028]
 [0.1352133  0.33047619 0.15332429 0.17077465 0.13184358]
 ...
 [0.24367317 0.45428571 0.40841248 0.69542254 0.23240223]
 [0.37093275 0.45238095 0.2890095  0.65492958 0.4603352 ]
 [0.37744035 0.49142857 0.4816825  0.78169014 0.47039106]]
[[0.22848879 0.45238095 0.55088195 0.69014085 0.29944134]
 [0.2812726  0.4952381  0.19402985 0.50352113 0.20893855]
 [0.11641359 0.27047619 0.12211669 0.26760563 0.22681564]
 ...
 [0.30730296 0.43904762 0.33649932 0.26760563 0.1877095 ]
 [0.15401302 0.29809524 0.18453189 0.25528169 0.16089385]
 [0.31019523 0.47714286 0.41112619 0.7165493  0.28156425]]
[[ True False False ... False False False]
 [False False  True ... False False False]
 [False False False ... False False False]
 ...
 [False  True False ... False False False]
 [False  True False ... False False False]
 [False False False ... False  True False]]


In [ ]:
print(testy_label)

# SVM

In [ ]:
for kernel in ['rbf','linear','poly']:
    for C in [1,10,100,1000]:
        if(kernel!='poly'):
            model = SVC(kernel=kernel,C=C)
            model.fit(trainx,trainy_label)
            print("Training set score for SVM: %f" % model.score(trainx, trainy_label))
            print("Testing  set score for SVM: %f" % model.score(testx, testy_label))
        else:
            for degree in [2,3,4,5]:
                model = SVC(kernel=kernel,C=C,degree=degree)
                model.fit(trainx,trainy_label)
                print("Training set score for SVM: %f" % model.score(trainx, trainy_label))
                print("Testing  set score for SVM: %f" % model.score(testx, testy_label))
        print('&&----------------------------&&',kernel,C)

In [ ]:
param_grid_svm = [
                {'kernel': ['rbf'], 'C': [500, 1000,2000,3000,5000,7000,10000]},
                #{'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
                #{'kernel': ['poly'], 'C': [1,10,100,1000], 'degree':[2,3,4,5]}
                 ]

In [ ]:
grid_search_svm = GridSearchCV(SVC(), param_grid_svm, cv=5, verbose=2,n_jobs=-1)

In [ ]:
grid_search_svm.fit(trainx,trainy_label)

In [ ]:
#View the accuracy score
print('Best score for training data:', grid_search_svm.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best C:',grid_search_svm.best_estimator_.C,"\n") 
print('Best Kernel:',grid_search_svm.best_estimator_.kernel,"\n")
print('Best Gamma:',grid_search_svm.best_estimator_.gamma,"\n")

In [ ]:
svm_final_model = grid_search_svm.best_estimator_
predictedy = svm_final_model.predict(testx)

In [ ]:
print("Training set score for SVM: %f" % svm_final_model.score(trainx, trainy_label))
print("Testing  set score for SVM: %f" % svm_final_model.score(testx, testy_label))

In [23]:
print(confusion_matrix(testy_label,predictedy))

NameError: name 'predictedy' is not defined

# Decision tree

In [ ]:
for criterion in ['gini', 'entropy','log_loss']:
    for max_depth in [3, 5, 10, None]:
        model = DecisionTreeClassifier(random_state=42,criterion=criterion,max_depth=max_depth)
        model.fit(trainx,trainy_label)
        print("Training set score for DT: %f" % model.score(trainx, trainy_label))
        print("Testing  set score for Dt: %f" % model.score(testx, testy_label))       
        print('&&----------------------------&&',criterion,max_depth)

In [ ]:
param_grid_dt = {
    'criterion': ['gini', 'entropy','log_loss'],     # Try both 'gini' and 'entropy'
    'max_depth': [None],        # Try different values of max_depth
    'min_samples_split': [2, 5, 10],      # Varying minimum samples required to split an internal node, default = 2
    'min_samples_leaf': [1, 2, 4],        # Varying minimum samples required to be at a leaf node, default = 1
}

In [ ]:
grid_search_dt = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), param_grid=param_grid_dt, cv=5, verbose=2, n_jobs=-1)

In [ ]:
grid_search_dt.fit(trainx,trainy_label)

In [ ]:
# View the accuracy score
print('Best score for training data:', grid_search_dt.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best criterion:',grid_search_dt.best_estimator_.criterion,"\n") 
print('Best max_depth:',grid_search_dt.best_estimator_.max_depth,"\n")
print('Best min_samples_split:',grid_search_dt.best_estimator_.min_samples_split,"\n") 
print('Best min_samples_leaf:',grid_search_dt.best_estimator_.min_samples_leaf,"\n")

In [ ]:
dt_final_model = grid_search_dt.best_estimator_
predictedy = dt_final_model.predict(testx)
print("Training set score for dt: %f" % dt_final_model.score(trainx, trainy_label))
print("Testing  set score for dt: %f" % dt_final_model.score(testx, testy_label))
print(confusion_matrix(testy_label,predictedy))

# Random Forest

In [ ]:
 for criterion in ['gini', 'entropy']: 
    for max_depth in [None, 10, 20, 30]:
        for n_estimators in [50, 100, 200]:
            model = RandomForestClassifier(random_state=42,criterion=criterion,max_depth=max_depth, n_estimators= n_estimators)
            model.fit(trainx,trainy_label)
            print("Training set score for DT: %f" % model.score(trainx, trainy_label))
            print("Testing  set score for Dt: %f" % model.score(testx, testy_label))       
            print('&&----------------------------&&',criterion,max_depth,n_estimators) 

In [ ]:
# model = RandomForestClassifier(random_state=42,min_samples_leaf=4)
# model.fit(trainx,trainy_label)
# print("Training set score for DT: %f" % model.score(trainx, trainy_label))
# print("Testing  set score for Dt: %f" % model.score(testx, testy_label))

In [ ]:
param_grid_rf = {
    'n_estimators': [70, 100, 150],      # Number of trees in the forest
    'max_depth': [None,30,40,50,70,100],     # Maximum depth of each tree
    'criterion': ['gini', 'entropy']     # Split criterion
}
grid_search_rf = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid_rf, cv=5, verbose=2, n_jobs=-1)

In [ ]:
grid_search_rf.fit(trainx,trainy_label)

In [ ]:
# View the accuracy score
print('Best score for training data:', grid_search_rf.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best criterion:',grid_search_rf.best_estimator_.criterion,"\n") 
print('Best max_depth:',grid_search_rf.best_estimator_.max_depth,"\n")
print('Best n_estimators:',grid_search_rf.best_estimator_.n_estimators,"\n")

In [ ]:
cv_result=pd.DataFrame(grid_search_rf.cv_results_)
print(cv_result[['param_criterion', 'param_max_depth', 'param_n_estimators', 
                 'mean_test_score', 'rank_test_score']])

In [ ]:
rf_final_model = grid_search_rf.best_estimator_
predictedy = rf_final_model.predict(testx)
print("Training set score for rf: %f" % rf_final_model.score(trainx, trainy_label))
print("Testing  set score for dt: %f" % rf_final_model.score(testx, testy_label))
print(confusion_matrix(testy_label,predictedy))

# KNN

In [ ]:
for metric in ['euclidean', 'manhattan']:
    for weights in ['uniform', 'distance']:
        for n_neighbors in [3, 5, 7, 9]:
            model = KNeighborsClassifier(metric=metric,weights=weights, n_neighbors=n_neighbors)
            model.fit(trainx,trainy_label)
            print("Training set score for knn: %f" % model.score(trainx, trainy_label))
            print("Testing  set score for knn: %f" % model.score(testx, testy_label))       
            print('&&----------------------------&&',metric,weights,n_neighbors)

In [ ]:
param_grid = {
    'n_neighbors': [1,2, 3, 5],            # Number of neighbors to consider
    'weights': ['uniform', 'distance'],     # Weight function (uniform or distance-based)
    'metric': ['euclidean', 'manhattan']    # Distance metric
}
grid_search_knn = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

In [ ]:
grid_search_knn.fit(trainx,trainy_label)

In [ ]:
# View the accuracy score
print('Best score for training data:', grid_search_knn.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best criterion:',grid_search_knn.best_estimator_.n_neighbors,"\n") 
print('Best max_depth:',grid_search_knn.best_estimator_.weights,"\n")
print('Best n_estimators:',grid_search_knn.best_estimator_.metric,"\n")

In [ ]:
knn_final_model = grid_search_knn.best_estimator_
predictedy = knn_final_model.predict(testx)
print("Training set score for knn: %f" % knn_final_model.score(trainx, trainy_label))
print("Testing  set score for knn: %f" % knn_final_model.score(testx, testy_label))
print(confusion_matrix(testy_label,predictedy))

# Deep Learning

In [ ]:
# First define baseline model. Then use it in Keras Classifier for the training
# BatchNorm really improves the accuracy of the model
def dl_model():
    # Create model here
    model = Sequential()
    model.add(Input(shape=(5,)))
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Dropout(0.1))
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(ReLU())
    #model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Dropout(0.1))
    model.add(Dense(64))
    model.add(BatchNormalization())
    model.add(ReLU())
    #model.add(Dropout(0.2))
    model.add(Dense(32))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Dense(16))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Dense(8, activation = 'softmax')) # Softmax for multi-class classification
    # Compile model here
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [ ]:
#we are using MinMax Scaled data for training here
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=150,         # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True,  # Restore model weights from the epoch with the best value of the monitored quantity
    verbose = 1
)
dl_final_model = dl_model()
history = dl_final_model.fit(dltrainx, trainy_hot, validation_split = 0.1, epochs = 400, verbose = 1, batch_size=32, callbacks=[early_stopping])   #default batch-size is 32

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training & Validation loss')
plt.xlabel('epochs')
plt.show()
print("batch_size = 32")
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training & Validation Accuracy')
plt.xlabel('epochs')
plt.show()
#orange is validation

In [ ]:
# Assuming you have your trained model 'dl_final_model' and evaluation data 'X_eval' and 'y_eval'
loss, accuracy = dl_final_model.evaluate(dltestx, testy_hot, verbose=1)

print(f"Evaluation Loss: {loss}")
print(f"Evaluation Accuracy: {accuracy}")

# best_epoch = np.argmax(history.history['val_accuracy'])
# best_accuracy = history.history['val_accuracy'][best_epoch]
# print(f"Best epoch: {best_epoch}")
# print(f"Best validation accuracy: {best_accuracy:.4f}")

In [ ]:
dl_model_wrapped = KerasClassifier(model = dl_model, epochs = 300, batch_size = 32, verbose = 1)  #for scikit-learn wrapper
kfold = KFold(n_splits = 5, shuffle = True)
results = cross_val_score(dl_model_wrapped, dltrainx, trainy_hot, cv = kfold) #will only work if the estimator model is wrapped in scikit-learn 
#this basically trains our model five times with different validation sets (taken like in cv=5)
#then averaging the accuracy we get for the 5 validation sets to get the final accuracy and std.
print("Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [53]:
from sklearn.linear_model import LogisticRegression

In [56]:
model=LogisticRegression()
#model.fit(trainx,trainy_label)

In [55]:
print("Training set score for knn: %f" % model.score(trainx, trainy_label))
print("Testing  set score for knn: %f" % model.score(testx, testy_label))       
#print('&&----------------------------&&',metric,weights,n_neighbors)

Training set score for knn: 0.517651
Testing  set score for knn: 0.508158


In [57]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],    # Regularization strength (Inverse of regularization)
    'penalty': ['l2'],               # L2 regularization (L1 is not supported with 'multinomial')
    'solver': ['newton-cg', 'lbfgs', 'saga'],  # Solvers supporting multinomial classification
    'max_iter': [100, 200, 300]      # Number of iterations for optimization
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1, scoring='accuracy')

In [58]:
grid_search.fit(trainx, trainy_label)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'max_iter': [100, 200, 300], 'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'saga']},
             scoring='accuracy', verbose=1)

In [60]:
# View the accuracy score
print('Best score for training data:', grid_search.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best criterion:',grid_search.best_estimator_.n_neighbors,"\n") 
print('Best max_depth:',grid_search.best_estimator_.weights,"\n")
print('Best n_estimators:',grid_search.best_estimator_.metric,"\n")

Best score for training data: 0.5168077689508231 



AttributeError: 'LogisticRegression' object has no attribute 'n_neighbors'

In [61]:
knn_final_model = grid_search.best_estimator_
predictedy = knn_final_model.predict(testx)
print("Training set score for knn: %f" % knn_final_model.score(trainx, trainy_label))
print("Testing  set score for knn: %f" % knn_final_model.score(testx, testy_label))
print(confusion_matrix(testy_label,predictedy))

Training set score for knn: 0.518746
Testing  set score for knn: 0.508944
[[172  18  77  77  57   1 133  84]
 [109 453   8 110  25   5  59   7]
 [ 43  30 490 164  21  17  12   7]
 [ 99  75 206 183  29   7  41  54]
 [ 37  11   2  84 257  39  32   8]
 [ 24   4   8  78  44 220   0   2]
 [ 51  43  10 140  48   0 397  50]
 [ 24  15  27  57  70  15   0 417]]
